# Synthea - Ambulatory Patients Analysis
Only the first encounter is of interest

## Libraries and Paths

In [11]:
from pathlib import Path
import shutil
import pandas as pd

# Absolute paths to files and directories
BASE_DIRECTORY = (Path.cwd()).parent
RAW_DATA_PATH = Path(BASE_DIRECTORY / 'data/raw/csv')
PROCESSED_DATA_PATH = Path(BASE_DIRECTORY / 'data/processed/ambulatory')
ANALYSIS_DATA_PATH = Path(BASE_DIRECTORY / 'data/analyzed')

## Data Processing

### Opening the Raw Data Files

In [12]:
# Get the raw data file names
raw_files = [file.name[:-4] for file in RAW_DATA_PATH.iterdir()]

# Load the DataFrames with raw data
dfs = dict()
for file in raw_files:
    dfs[file] = pd.read_csv(f'{RAW_DATA_PATH}/{file}.csv')

### Finding the Ambulatory Patients

In [13]:
# Drop unnecessary encounter columns
dfs['encounters'] = dfs['encounters'] \
    .drop(columns=[
        'ORGANIZATION', 'PROVIDER', 'PAYER', 'BASE_ENCOUNTER_COST',
        'TOTAL_CLAIM_COST', 'PAYER_COVERAGE'
    ])

# Filter only emergency encounters
dfs['encounters'] = dfs['encounters'] \
    .query('ENCOUNTERCLASS == "ambulatory"') \
    .query('REASONDESCRIPTION.notnull()') \
    .reset_index(drop=True)

# Get the identifiers (encounters and patients)
encounters_ids = dfs['encounters']['Id'].tolist()
patients_ids = dfs['encounters']['PATIENT'].tolist()

# Print some of the filtered encounters data
dfs['encounters']

,Id,START,STOP,PATIENT,ENCOUNTERCLASS,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,5765ddf9-34e5-017b-e84e-95b870f1c6db,2016-01-02T14:20:29Z,2016-01-02T14:35:29Z,54323f01-0951-8e76-f4ec-2c72f8f92673,ambulatory,185345009,Encounter for symptom,43878008.0,Streptococcal sore throat (disorder)
1,0e3811c2-f81b-08ba-7858-d0f6bcc9ab25,2017-06-08T00:20:29Z,2017-06-08T00:35:29Z,54323f01-0951-8e76-f4ec-2c72f8f92673,ambulatory,185345009,Encounter for symptom,43878008.0,Streptococcal sore throat (disorder)
2,e0de6e09-d8d4-542e-dbf7-bc289d8934f6,2019-11-15T05:19:29Z,2019-11-15T05:34:29Z,54323f01-0951-8e76-f4ec-2c72f8f92673,ambulatory,185349003,Encounter for check up,58150001.0,Fracture of clavicle
3,84631d66-6172-7d9b-3b3a-f1628a142c37,2021-12-21T12:20:29Z,2021-12-21T12:35:29Z,54323f01-0951-8e76-f4ec-2c72f8f92673,ambulatory,185345009,Encounter for symptom,444814009.0,Viral sinusitis (disorder)
4,693b3025-2d2e-146d-2bb5-197b2ae5bf2c,2019-06-04T21:53:33Z,2019-06-04T22:08:33Z,38ec7e85-d9c7-cfbb-3292-9221b528f77a,ambulatory,185345009,Encounter for symptom,10509002.0,Acute bronchitis (disorder)
...,...,...,...,...,...,...,...,...,...
393604,930f7970-fd40-aa2f-f98b-63c141bb88d1,2020-08-29T17:01:18Z,2020-08-29T17:16:18Z,b7b3a619-833e-035d-1668-edc7cea0378a,ambulatory,185349003,Encounter for check up,16114001.0,Fracture of ankle
393605,34d6bf4f-5882-08e9-bf2c-9b26fab9da2c,2020-12-10T15:37:32Z,2020-12-10T15:52:32Z,b7b3a619-833e-035d-1668-edc7cea0378a,ambulatory,390906007,Follow-up encounter,55822004.0,Hyperlipidemia
393606,403a1a58-1448-bbfe-0f56-ffd88f5ed293,2021-12-10T15:37:32Z,2021-12-10T15:52:32Z,b7b3a619-833e-035d-1668-edc7cea0378a,ambulatory,390906007,Follow-up encounter,55822004.0,Hyperlipidemia
393607,ecf7e6ca-3b39-69db-8cca-11081aa1f998,2022-12-22T15:37:32Z,2022-12-22T15:52:32Z,b7b3a619-833e-035d-1668-edc7cea0378a,ambulatory,390906007,Follow-up encounter,55822004.0,Hyperlipidemia


### Filtering the Data Associated with the Ambulatory Encounters

In [14]:
# Filter data from DataFrames with encounter as foreign key
for file in raw_files:
    if (file != 'encounters') and (file != 'patients'):
        dfs[file] = dfs[file] \
            .query('ENCOUNTER in @encounters_ids') \
            .reset_index(drop=True)

# # Drop unnecessary columns from immunizations, medications and procedures DataFrames
dfs['immunizations'] = dfs['immunizations'].drop(columns=['BASE_COST'])
dfs['medications'] = dfs['medications'] \
    .drop(columns=['BASE_COST', 'PAYER_COVERAGE', 'TOTALCOST'])
dfs['procedures'] = dfs['procedures'].drop(columns=['BASE_COST'])

# Filter data and drop unnecessary columns from the patients DataFrame
dfs['patients'] = dfs['patients'] \
    .drop(columns=['HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'INCOME']) \
    .query('Id in @patients_ids') \
    .reset_index(drop=True)

### Writing CSV Files with the Processed Data

In [15]:
# Verify if the processed data directory exists
if PROCESSED_DATA_PATH.exists():
    # Delete the directory
    shutil.rmtree(PROCESSED_DATA_PATH)

# Create the processed data directory
PROCESSED_DATA_PATH.mkdir(parents=True)

# Write CSV files with processed data
for file in raw_files:
    dfs[file].to_csv(f'{PROCESSED_DATA_PATH}/{file}.csv', index=False)

## Data Analysis

### Treating the Encounters DataFrame

In [16]:
# Open the encounters data file
df_encounters = pd.read_csv(f'{PROCESSED_DATA_PATH}/encounters.csv')

# Select the necessary encounter columns
df_encounters_reason = df_encounters[['Id', 'REASONDESCRIPTION']].copy()

# Rename the remaining columns
df_encounters_reason = df_encounters_reason.rename(
    columns={'Id': 'ENCOUNTER', 'REASONDESCRIPTION': 'REASON'}
)

# Print some of the encounters data
df_encounters_reason

,ENCOUNTER,REASON
0,5765ddf9-34e5-017b-e84e-95b870f1c6db,Streptococcal sore throat (disorder)
1,0e3811c2-f81b-08ba-7858-d0f6bcc9ab25,Streptococcal sore throat (disorder)
2,e0de6e09-d8d4-542e-dbf7-bc289d8934f6,Fracture of clavicle
3,84631d66-6172-7d9b-3b3a-f1628a142c37,Viral sinusitis (disorder)
4,693b3025-2d2e-146d-2bb5-197b2ae5bf2c,Acute bronchitis (disorder)
...,...,...
393604,930f7970-fd40-aa2f-f98b-63c141bb88d1,Fracture of ankle
393605,34d6bf4f-5882-08e9-bf2c-9b26fab9da2c,Hyperlipidemia
393606,403a1a58-1448-bbfe-0f56-ffd88f5ed293,Hyperlipidemia
393607,ecf7e6ca-3b39-69db-8cca-11081aa1f998,Hyperlipidemia


### Verifying the Number of Conditions per Encounter

In [17]:
# Open the conditions data file
df_conditions = pd.read_csv(f'{PROCESSED_DATA_PATH}/conditions.csv')

# Aggregate conditions data using encounter codes
df_conditions_agg = df_conditions \
    .groupby(by=['ENCOUNTER'], as_index=False) \
    .agg(
        NUM_CONDITIONS=('CODE', pd.Series.nunique),
        CONDITIONS=('DESCRIPTION', 'unique')
    ) \
    .sort_values(by=['NUM_CONDITIONS'], ascending=False) \
    .reset_index(drop=True)

# Write a CSV file with the resulting DataFrame
df_conditions_agg.to_csv(f'{ANALYSIS_DATA_PATH}/ambulatory_conditions.csv', index=False)

# Print some of the aggregation result
df_conditions_agg

,ENCOUNTER,NUM_CONDITIONS,CONDITIONS
0,758c98fd-7774-0ca4-acc8-3cfd48b3aea3,11,"[Cerebral palsy (disorder), Pain (finding), Sp..."
1,6c64f250-38f1-cc81-c33e-8ffbfad84e2c,11,"[Nasal congestion (finding), Sputum finding (f..."
2,cba5bb3b-95c1-7428-e736-bd1b52234dee,11,"[Cough (finding), Sore throat symptom (finding..."
3,47a18299-5104-8145-9345-1f4f4c1d3f38,10,"[Cerebral palsy (disorder), Pain (finding), Po..."
4,8872f00f-fdc0-fb5c-c37e-e185e584043c,10,"[Cough (finding), Sputum finding (finding), Fa..."
...,...,...,...
47818,577f20b5-4350-a4b7-f2f1-c196a70d3693,1,[Viral sinusitis (disorder)]
47819,577f30c5-f89b-bab0-6bd1-ea69c2dfdc0e,1,[Normal pregnancy]
47820,5782bb3a-7a08-2bde-a336-f152480f77a0,1,[Acute bronchitis (disorder)]
47821,5783fe91-bf19-a439-a043-5701cf80f616,1,[Viral sinusitis (disorder)]


### Verifying the Number of Observations per Encounter

In [18]:
# Open the observations data file
df_observations = pd.read_csv(f'{PROCESSED_DATA_PATH}/observations.csv')

# Aggregate observations data using encounter codes
df_observations_agg = df_observations \
    .groupby(by=['ENCOUNTER'], as_index=False) \
    .agg(
        NUM_CATEGORIES=('CATEGORY', pd.Series.nunique),
        CATEGORIES=('CATEGORY', 'unique'),
        NUM_OBSERVATIONS=('CODE', pd.Series.nunique),
        OBSERVATIONS=('DESCRIPTION', 'unique')
    ) \
    .sort_values(by=['NUM_CATEGORIES', 'NUM_OBSERVATIONS'], ascending=False) \
    .reset_index(drop=True)

# Write a CSV file with the resulting DataFrame
df_observations_agg.to_csv(f'{ANALYSIS_DATA_PATH}/ambulatory_observations.csv', index=False)

# Print some of the aggregation result
df_observations_agg

,ENCOUNTER,NUM_CATEGORIES,CATEGORIES,NUM_OBSERVATIONS,OBSERVATIONS
0,c044abca-d646-a60e-f5c4-0c8b4c7323e5,4,"[procedure, survey, exam, laboratory]",30,"[US Guidance for biopsy of Prostate, Within th..."
1,ef56b8af-559a-5ab3-4712-ded79f5c2f98,4,"[procedure, survey, exam, laboratory]",30,"[US Guidance for biopsy of Prostate, Within th..."
2,01b5ed55-1e53-a20b-2f97-42b6d9aae7e8,4,"[procedure, survey, exam, laboratory]",28,"[US Guidance for biopsy of Prostate, Within th..."
3,4923addd-b048-0af1-9b32-a3758576ef56,4,"[procedure, survey, exam, laboratory]",28,"[US Guidance for biopsy of Prostate, Within th..."
4,b296d369-5d84-7ca0-4a95-71292298cec6,4,"[procedure, survey, exam, laboratory]",28,"[US Guidance for biopsy of Prostate, Within th..."
...,...,...,...,...,...
276239,ffe94c49-3707-358b-9093-c77347d03df5,1,[survey],1,[Pain severity - Reported]
276240,ffea6409-bafb-5bc2-d964-37b5244cad8e,1,[survey],1,[Pain severity - Reported]
276241,fff65be7-9944-0f77-148a-567982d51dc7,1,[therapy],1,[Mental health Outpatient Note]
276242,fff6cdab-c1c3-9e8c-5ced-fa3c6073837c,1,[vital-signs],1,[Body temperature]


### Joining Encounters, Conditions and Observations Data

In [19]:
# Merge encounters reason and aggregation of conditions and observations
df_merged_data = df_encounters_reason \
    .merge(right=df_conditions_agg, how='left', on='ENCOUNTER') \
    .merge(right=df_observations_agg, how='left', on='ENCOUNTER') \
    .fillna({
        'NUM_CONDITIONS': 0,
        'CONDITIONS': '',
        'NUM_CATEGORIES': 0,
        'CATEGORIES': '',
        'NUM_OBSERVATIONS': 0,
        'OBSERVATIONS': ''
    }) \
    .astype(
        {'NUM_CONDITIONS': int, 'NUM_CATEGORIES': int, 'NUM_OBSERVATIONS': int}
    )

# Print some of the merged data
df_merged_data

,ENCOUNTER,REASON,NUM_CONDITIONS,CONDITIONS,NUM_CATEGORIES,CATEGORIES,NUM_OBSERVATIONS,OBSERVATIONS
0,5765ddf9-34e5-017b-e84e-95b870f1c6db,Streptococcal sore throat (disorder),1,[Streptococcal sore throat (disorder)],1,[vital-signs],1,[Body temperature]
1,0e3811c2-f81b-08ba-7858-d0f6bcc9ab25,Streptococcal sore throat (disorder),1,[Streptococcal sore throat (disorder)],1,[vital-signs],1,[Body temperature]
2,e0de6e09-d8d4-542e-dbf7-bc289d8934f6,Fracture of clavicle,0,,0,,0,
3,84631d66-6172-7d9b-3b3a-f1628a142c37,Viral sinusitis (disorder),1,[Viral sinusitis (disorder)],0,,0,
4,693b3025-2d2e-146d-2bb5-197b2ae5bf2c,Acute bronchitis (disorder),1,[Acute bronchitis (disorder)],0,,0,
...,...,...,...,...,...,...,...,...
393604,930f7970-fd40-aa2f-f98b-63c141bb88d1,Fracture of ankle,0,,0,,0,
393605,34d6bf4f-5882-08e9-bf2c-9b26fab9da2c,Hyperlipidemia,0,,1,[laboratory],20,"[Glucose [Mass/volume] in Blood, Urea nitrogen..."
393606,403a1a58-1448-bbfe-0f56-ffd88f5ed293,Hyperlipidemia,0,,1,[laboratory],20,"[Glucose [Mass/volume] in Blood, Urea nitrogen..."
393607,ecf7e6ca-3b39-69db-8cca-11081aa1f998,Hyperlipidemia,0,,1,[laboratory],20,"[Glucose [Mass/volume] in Blood, Urea nitrogen..."


### Metrics of Conditions and Observations per Encounter Reason

In [20]:
# Calculate the metrics of conditions and observations per encounter reason
df_metrics = df_merged_data \
    .groupby(by=['REASON'], as_index=False) \
    .agg(
        CASES=('ENCOUNTER', 'count'),
        MIN_CONDS=('NUM_CONDITIONS', 'min'),
        MAX_CONDS=('NUM_CONDITIONS', 'max'),
        MEDIAN_CONDS=('NUM_CONDITIONS', 'median'),
        MEAN_CONDS=('NUM_CONDITIONS', 'mean'),
        STD_CONDS=('NUM_CONDITIONS', 'std'),
        MIN_OBS=('NUM_OBSERVATIONS', 'min'),
        MAX_OBS=('NUM_OBSERVATIONS', 'max'),
        MEDIAN_OBS=('NUM_OBSERVATIONS', 'median'),
        MEAN_OBS=('NUM_OBSERVATIONS', 'mean'),
        STD_OBS=('NUM_OBSERVATIONS', 'std')
    ) \
    .fillna({'STD_CONDS': 0,'STD_OBS': 0}) \
    .astype({'MEDIAN_CONDS': int, 'MEDIAN_OBS': int}) \
    .sort_values(by=['MEDIAN_CONDS', 'MEDIAN_OBS'], ascending=False) \
    .reset_index(drop=True)

# Write a CSV file with the resulting DataFrame
df_metrics.to_csv(f'{ANALYSIS_DATA_PATH}/ambulatory_metrics.csv', index=False)

# Print some of the metrics data
df_metrics

,REASON,CASES,MIN_CONDS,MAX_CONDS,MEDIAN_CONDS,MEAN_CONDS,STD_CONDS,MIN_OBS,MAX_OBS,MEDIAN_OBS,MEAN_OBS,STD_OBS
0,Suspected COVID-19,939,1,11,5,5.010650,1.730787,10,19,17,14.340788,3.402142
1,Neoplasm of prostate,250,0,5,2,1.596000,1.144563,0,71,44,33.020000,24.781041
2,Bleeding from anus,58,0,2,2,1.258621,0.909219,0,0,0,0.000000,0.000000
3,Abnormal findings diagnostic imaging heart+cor...,691,0,1,1,0.684515,0.465045,0,52,13,19.450072,21.613168
4,Screening for malignant neoplasm of breast (pr...,95,1,1,1,1.000000,0.000000,0,13,11,6.831579,5.745771
...,...,...,...,...,...,...,...,...,...,...,...,...
106,Soy bean,71,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000
107,Suicidal deliberate poisoning,2,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000
108,Third degree burn,9,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000
109,Tree nut (substance),136,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000
